## 개요

데이터 설명으로부터 파일 이름은 다음과 같은 내용을 포함함을 확인할 수 있다.

* ID
* Record time
* Record_mobile_version
* gender
* age(week)
* status

예를 들어 0D1AD73E-4C5E-45F3-85C4-9A3CB71E8856-1430742197-1.0-m-04-hu.caf 파일명은 아래와 같이 해석될 수 있다.

- 샘플은 고유 ID가 0D1AD73E-4C5E-45F3-85C4-9A3CB71E8856인 앱 인스턴스로 기록되었다. 이 ID는 설치 시 생성되므로 장치나 사용자가 아닌 설치된 인스턴스를 식별한다.
- 기록은 1430742197(유닉스 시간 에포크)에 이루어졌으며 이는 2015년 5월 4일 월요일 12:23:17 GMT로 변환된다.
- 모바일 앱 버전 1.0 사용함.
- gender는 male.
- 아기는 사용자에 따라 생후 0-4주.
- 울음의 의심되는 이유는 배고픔.

자세한 변수에 대한 설명은 아래와 같다.

* **Gender**
  - *m* - male
  - *f* - female

* **Age**
  - *04* - 0 to 4 weeks old
  - *48* - 4 to 8 weeks old
  - *26* - 2 to 6 months old
  - *72* - 7 month to 2 years old
  - *22* - more than 2 years old

* **Reason**
  - *hu* - hungry
  - *bu* - needs burping
  - *bp* - belly pain
  - *dc* - discomfort
  - *ti* - tired


이를 기반으로 파일에 대한 정보를 담고 있는 csv 파일을 만들어보자.

In [1]:
# Load packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shutil import move

In [2]:
# OS setup
main_path = os.getcwd()
data_path = os.path.join(main_path, 'data')
origin_file_path = "/Users/jaewone/Downloads/donateacry_corpus_cleaned_and_updated_data"
csv_path = os.path.join(main_path, 'info.csv')

label_list = ['belly_pain','discomfort','burping','tired','hungry']

In [3]:
# Check counts of file of label
count_total = 0
for label in label_list:
    label_file_path = os.path.join(origin_file_path, label)
    count = len(os.listdir(label_file_path))
    count_total += count
    print(f'{label}: {count}')

print()
print(f'Count total: {count_total}')

belly_pain: 16
discomfort: 27
burping: 8
tired: 24
hungry: 382

Count total: 457


In [4]:
# Extract data
df_columns = ['gender', 'age', 'state', 'file']

def get_data_df(label):
    files_path = os.path.join(origin_file_path, label)
    names = [name.rsplit('.',1)[0].rsplit('-', 3)[1:] + [os.path.join(origin_file_path, label, name)] for name in os.listdir(files_path)]
    return pd.DataFrame(names, columns=df_columns)

df = pd.DataFrame(columns=df_columns)
for label in label_list:
    df = pd.concat([df, get_data_df(label)])

df['source'] = 'donate_a_cry'

df.tail(3)

,gender,age,state,file,source
379,m,04,hu,/Users/jaewone/Downloads/donateacry_corpus_cle...,donate_a_cry
380,m,72,hu,/Users/jaewone/Downloads/donateacry_corpus_cle...,donate_a_cry
381,f,26,hu,/Users/jaewone/Downloads/donateacry_corpus_cle...,donate_a_cry


In [5]:
# append to csv
origin_df = pd.read_csv(csv_path, index_col=0)
origin_df = pd.concat([origin_df, df])
origin_df.tail(3)

,state,gender,age,source,file
379,hu,m,04,donate_a_cry,/Users/jaewone/Downloads/donateacry_corpus_cle...
380,hu,m,72,donate_a_cry,/Users/jaewone/Downloads/donateacry_corpus_cle...
381,hu,f,26,donate_a_cry,/Users/jaewone/Downloads/donateacry_corpus_cle...


In [6]:
# move file to data folder
def move_all_file_in_folder(from_path, to_path, extension=None):
  for (path, dirs, files) in os.walk(from_path):
    for dir in dirs:
      move_all_file_in_folder(os.path.join(from_path, dir), to_path, extension)
    for file in files:
      if extension:
        s = file.rsplit('.', 1)
        if len(s) == 1 or f'.{s[1]}' not in extension:
          continue
      move(os.path.join(path, file), os.path.join(to_path, file))

move_all_file_in_folder(origin_file_path, data_path, extension=['.wav'])

In [7]:
# Update file path in datframe
origin_df['file'] = origin_df['file'].str.replace(origin_file_path, '').str.replace(
    '|'.join([f'/{label}/' for label in label_list]), '')

# Update state name
# ['belly_pain','discomfort','burping','tired','hungry']
def get_full_state_name(state):
    if state == "bp":
        return 'belly_pain' 
    if state == "dc":
        return 'discomfort' 
    if state == "bu":
        return 'burping' 
    if state == "ti":
        return 'tired' 
    if state == "hu":
        return 'hungry'
    # if state == "ch":
    return state

origin_df['state'] = origin_df['state'].apply(get_full_state_name)

origin_df.tail(3)

/var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/ipykernel_62716/212709108.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  origin_df['file'] = origin_df['file'].str.replace(origin_file_path, '').str.replace(


,state,gender,age,source,file
379,hungry,m,04,donate_a_cry,F24DE44B-762C-4149-AC92-96A5E57ED118-143081691...
380,hungry,m,72,donate_a_cry,bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-143063764...
381,hungry,f,26,donate_a_cry,ca7d8d54-13cb-4060-8db4-90651b60c078-143467476...


In [8]:
# Update csv
origin_df.to_csv(csv_path)

In [9]:
# 문서에서 제시된 라벨링으로 데이터가 입력되었음을 확인 할 수 있다.
for col in df.columns[:-2]:
    print(f'Column: {col}')
    for val in df[col].unique():
        print(f'Value "{val}" with counts {len(df[df[col]==val])}')
    print()

Column: gender
Value "m" with counts 276
Value "f" with counts 181

Column: age
Value "26" with counts 162
Value "04" with counts 149
Value "72" with counts 81
Value "48" with counts 55
Value "22" with counts 10

Column: state
Value "bp" with counts 16
Value "dc" with counts 20
Value "ch" with counts 7
Value "bu" with counts 8
Value "ti" with counts 24
Value "hu" with counts 382

